In [1]:
import pandas as pd
from alphabet_detector import AlphabetDetector
import string
import re
import nltk
import numpy as np
import operator
from nltk.stem.snowball import SnowballStemmer

# Data cleaning

In [2]:
path_to_data = '../data/comments1.csv'

In [3]:
df = pd.read_csv(path_to_data)
df = df.dropna()

In [4]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [5]:
def clean(x):
    ad = AlphabetDetector()
    res = x
    for ch in string.punctuation:                                                                                                     
        res = res.replace(ch, ' ')
    res = ''.join([i for i in res if not i.isdigit()])
    res = res.lower()
    res = emoji_pattern.sub(r' ', res)
    res = res.replace('\n', ' ')
    res = res.replace('\t', ' ')
    res = res.replace('\ufeff', ' ')
    res = res.replace('\r\n', '  ')
    res = res.replace('\xa0', ' ')
    res = res.replace('ё', 'е')
    res = re.sub(' +',' ', res)
    if  not ad.only_alphabet_chars(res, "CYRILLIC"): 
        res = ''
    return res

In [6]:
df.text = df.text.apply(clean)
df = df[df.text != '']

In [7]:
#df.to_csv('../data/comments_clean1.csv', index=False)

In [8]:
#remove comments that were manualy labeld
manual = pd.read_csv('../data/manual.csv')
df = df[~df.id.isin(manual.id)]

# Stemming bad words

In [13]:
def stem(word):
    stemmer = SnowballStemmer("russian", ignore_stopwords=True) 
    stemmed_word = stemmer.stem(word)
    if len(stemmed_word) <= 2:
        return word
    return stemmed_word

In [14]:
path_to_bad_words = '../data/bad_words.txt'

In [15]:
bad_words = open(path_to_bad_words).read().split('\n')[:-1]

In [16]:
report_dictionary = [] # array that will contain all bad words
stemmed_dictionary = [] # array that will contain actual stems used for finding bad words

In [17]:
for word in bad_words:
    report_dictionary.append(word)
    stemmed_dictionary.append(stem(word))

In [18]:
stemmed_dictionary = [i for i in stemmed_dictionary if len(i) > 2] 

In [19]:
stemmed_dictionary = list(set(stemmed_dictionary))

In [20]:
stemmed_dictionary.append('хуй') # nltk stemmer can not corectly stem word хуй
stemmed_dictionary.append('хуе')

# label coments with bad words

In [21]:
def label(x):
    global stemmed_dictionary
    tokens = nltk.word_tokenize(x)
    for bad_word in stemmed_dictionary:
        for token in tokens:
            if bad_word in token:
                return True
    return False

In [22]:
df['label'] = df.text.apply(label)

# Iterative process of finding new bad words

## Word stemming and counting

In [28]:
all_stemmed = dict()
for sentence in df.text:
    for token in nltk.word_tokenize(sentence):
        if token not in all_stemmed:
            all_stemmed[token] = stem(token)

In [29]:
all_counts = dict()
for sentence in df.text:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        word = stem(token)
        if word in all_counts:
            all_counts[word] += 1
        else:
            all_counts[word] = 1

# Likelihood of words being in comments labeld as bad or labeld as not bad

In [30]:
def likelihood(label):
    labeld_part = dict()
    part = df[df.label == label]
    for index, row in part.iterrows():
        sentence = row.text
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            stemed = stem(token)
            if stemed != '':
                if all_counts[stemed] >= 20:
                    if stemed in labeld_part:
                        labeld_part[stemed] += 1.0/len(part)
                    else:
                        labeld_part[stemed] = 1.0/len(part)
    return labeld_part

## Criteria functions for decision if word is bad or not bad

In [31]:
def relative_distance(p_good, p_bad):
    difference = dict()
    for key in p_good:
        if key in p_bad:
            difference[key] =(p_bad[key] - p_good[key])/ np.maximum( p_bad[key], p_good[key]) 
    return difference

In [32]:
def logg_odds(p_good, p_bad):
    ratio = dict()
    for key in p_good:
        if key in p_bad:
            odds_good =  p_good[key]/(1 -  p_good[key])
            odds_bad = p_bad[key]/(1- p_bad[key])
            ratio[key] = np.log(odds_bad/odds_good)
    return ratio

## Detecting new bad words

In [33]:
def unstem(stem):
    res = []
    for key in all_stemmed:
        if all_stemmed[key] == stem:
            res.append(key)
    return res    

In [34]:
def sort(x, rev = True):
    return sorted(x.items(), key=operator.itemgetter(1), reverse=rev)

In [35]:
def update_dictionary(word):
    global stemmed_dictionary
    if len(word) <= 2:
        return False
    #for key in stemmed_dictionary:
        #if word in key:
            #return False
    stemmed_dictionary.append(word)
    return True

In [36]:
def iterate(df, report_dictionary, stemmed_dictionary):
    iter = True
    while(iter):
        print('-----------------------------------')
        print('labeld as abusive: ', len(df[df.label == True]))
        #print('List of bad_words')
        #print(report_dictionary)
        p_bad = likelihood(True)
        p_good = likelihood(False)
        prob = relative_distance(p_good, p_bad)
        print(sort(prob)[:10])
        print('New bad words')
        new_words = []
        for key in prob:
            if prob[key] > 0.9:
                if update_dictionary(key):
                    new_words += unstem(key)
                    print( unstem(key))
        report_dictionary += new_words
        if len(new_words) == 0:
            iter = False
            break
        #print('\nNew dictionary')
        #print(report_dictionary)
        df['label'] =  df.text.apply(label)
        print('relabeld as abusive: ', len(df[df.label == True]))

In [37]:
iterate(df, report_dictionary, stemmed_dictionary)

-----------------------------------
labeld as abusive:  13139
[('чао', 0.9804790282154483), ('засад', 0.9795029796262208), ('закупа', 0.9627326902294923), ('колхоз', 0.9544510658360461), ('иностранц', 0.9471044635515374), ('дыряв', 0.935272567240697), ('овец', 0.935272567240697), ('звон', 0.9344095348039062), ('инфраструктур', 0.9316765987540689), ('хохлятск', 0.9276575751513672)]
New bad words
['лишение', 'лишенное', 'лишенным', 'лишения', 'лишенные', 'лишенный', 'лишенная', 'лишенными', 'лишень', 'лишении']
['дрался', 'дрались', 'драли', 'драл']
['дырявых', 'дырявые', 'дырявым', 'дырявого', 'дырявая', 'дырявое', 'дырявый']
['типы', 'типам', 'типах', 'типа', 'типи', 'тип', 'типу', 'типов', 'типо']
['звоните', 'звонил', 'звонили', 'звоним', 'звонишь', 'звонят', 'звонить', 'звонит', 'звони', 'звон', 'звонила']
['закупаются', 'закупаем', 'закупает', 'закупать', 'закупается', 'закупают']
['овец']
['чао']
['колхозы', 'колхозу', 'колхозах', 'колхоз', 'колхоза', 'колхозов', 'колхозам']
['алк

# Evaluation

In [38]:
manual['evaluation'] = manual.text.apply(label)

In [39]:
tp = 0
tn = 0
fp = 0
fn = 0
for index, row in manual.iterrows():
    if row.label == True and row.evaluation == True:
        tp += 1
    if row.label == False and row.evaluation == False:
        tn += 1
    if row.label == False and row.evaluation == True:
        fp += 1
    if row.label == True and row.evaluation == False:
        #print(row.text)
        fn += 1
accuracy = (tp + fp)/(tp + fp + fn + fp)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1  = 2 * (precision * recall)/(precision + recall)
print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

accuracy:  0.6294117647058823
precision:  0.7445482866043613
recall:  0.6907514450867052
f1:  0.7166416791604198
